# Introduction

In this tutorial, we will go through an example to update a preexisting model. This might be useful when you come across additional data that you would want to consider, without having to train a model from scratch.

The main abstraction that Lightwood offers for this is the `BaseMixer.partial_fit()` method. To call it, you need to pass new training data and a held-out dev subset for internal mixer usage (e.g. early stopping). If you are using an aggregate ensemble, it's likely you will want to do this for every single mixer. The convienient `PredictorInterface.adjust()` does this automatically for you.


# Initial model training

First, let's train a Lightwood predictor for the `concrete strength` dataset:

In [1]:
from lightwood.api.high_level import ProblemDefinition, json_ai_from_problem, predictor_from_json_ai
import pandas as pd

[nltk_data] Downloading package punkt to /home/runner/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
INFO:lightwood-2296:No torchvision detected, image helpers not supported.
INFO:lightwood-2296:No torchvision/pillow detected, image encoder not supported


In [2]:
# Load data
df = pd.read_csv('https://raw.githubusercontent.com/mindsdb/lightwood/staging/tests/data/concrete_strength.csv')

df = df.sample(frac=1, random_state=1)
train_df = df[:int(0.1*len(df))]
update_df = df[int(0.1*len(df)):int(0.8*len(df))]
test_df = df[int(0.8*len(df)):]

print(f'Train dataframe shape: {train_df.shape}')
print(f'Update dataframe shape: {update_df.shape}')
print(f'Test dataframe shape: {test_df.shape}')

Train dataframe shape: (103, 10)
Update dataframe shape: (721, 10)
Test dataframe shape: (206, 10)


Note that we have three different data splits.

We will use the `training` split for the initial model training. As you can see, it's only a 20% of the total data we have. The `update` split will be used as training data to adjust/update our model. Finally, the held out `test` set will give us a rough idea of the impact our updating procedure has on the model's predictive capabilities.

In [3]:
# Define predictive task and predictor
target = 'concrete_strength'
pdef = ProblemDefinition.from_dict({'target': target, 'time_aim': 200})
jai = json_ai_from_problem(df, pdef)

# We will keep the architecture simple: a single neural mixer, and a `BestOf` ensemble:
jai.model = {
    "module": "BestOf",
    "args": {
        "args": "$pred_args",
        "accuracy_functions": "$accuracy_functions",
        "submodels": [{
            "module": "Neural",
            "args": {
                "fit_on_dev": False,
                "stop_after": "$problem_definition.seconds_per_mixer",
                "search_hyperparameters": False,
            }
        }]
    }
}

# Build and train the predictor
predictor = predictor_from_json_ai(jai)
predictor.learn(train_df)

INFO:type_infer-2296:Analyzing a sample of 979
INFO:type_infer-2296:from a total population of 1030, this is equivalent to 95.0% of your data.
INFO:type_infer-2296:Infering type for: id
INFO:type_infer-2296:Column id has data type integer
INFO:type_infer-2296:Infering type for: cement
INFO:type_infer-2296:Column cement has data type float
INFO:type_infer-2296:Infering type for: slag
INFO:type_infer-2296:Column slag has data type float
INFO:type_infer-2296:Infering type for: flyAsh
INFO:type_infer-2296:Column flyAsh has data type float
INFO:type_infer-2296:Infering type for: water
INFO:type_infer-2296:Column water has data type float
INFO:type_infer-2296:Infering type for: superPlasticizer
INFO:type_infer-2296:Column superPlasticizer has data type float
INFO:type_infer-2296:Infering type for: coarseAggregate
INFO:type_infer-2296:Column coarseAggregate has data type float
INFO:type_infer-2296:Infering type for: fineAggregate
INFO:type_infer-2296:Column fineAggregate has data type float
I

In [4]:
# Train and get predictions for the held out test set
predictions = predictor.predict(test_df)
predictions

INFO:dataprep_ml-2296:[Predict phase 1/4] - Data preprocessing
INFO:dataprep_ml-2296:Cleaning the data
DEBUG:lightwood-2296: `preprocess` runtime: 0.02 seconds
INFO:dataprep_ml-2296:[Predict phase 2/4] - Feature generation
INFO:dataprep_ml-2296:Featurizing the data
DEBUG:lightwood-2296: `featurize` runtime: 0.05 seconds
INFO:dataprep_ml-2296:[Predict phase 3/4] - Calling ensemble
INFO:dataprep_ml-2296:[Predict phase 4/4] - Analyzing output
INFO:lightwood-2296:The block ICP is now running its explain() method
INFO:lightwood-2296:The block ConfStats is now running its explain() method
INFO:lightwood-2296:ConfStats.explain() has not been implemented, no modifications will be done to the data insights.
INFO:lightwood-2296:The block AccStats is now running its explain() method
INFO:lightwood-2296:AccStats.explain() has not been implemented, no modifications will be done to the data insights.
INFO:lightwood-2296:The block PermutationFeatureImportance is now running its explain() method
INFO:

,original_index,prediction,confidence,lower,upper
0,0,40.909611,0.9991,0.000000,87.398061
1,1,19.146826,0.9991,0.000000,65.635276
2,2,22.482285,0.9991,0.000000,68.970735
3,3,19.593748,0.9991,0.000000,66.082199
4,4,31.724546,0.9991,0.000000,78.212997
...,...,...,...,...,...
201,201,50.553104,0.9991,4.064654,97.041555
202,202,48.580437,0.9991,2.091986,95.068887
203,203,30.114182,0.9991,0.000000,76.602632
204,204,25.675997,0.9991,0.000000,72.164447


## Updating the predictor

For this, we have two options:

### `BaseMixer.partial_fit()`

Updates a single mixer. You need to pass the new data wrapped in `EncodedDs` objects.

**Arguments:** 
* `train_data: EncodedDs`
* `dev_data: EncodedDs`
* `adjust_args: Optional[dict]` - This will contain any arguments needed by the mixer to adjust new data.

If the mixer does not need a `dev_data` partition, pass a dummy:

```
dev_data = EncodedDs(predictor.encoders, pd.DataFrame(), predictor.target)
```

### `PredictorInterface.adjust()`

Updates all mixers inside the predictor by calling their respective `partial_fit()` methods. Any `adjust_args` will be transparently passed as well.

**Arguments:**

* `new_data: pd.DataFrame`
* `old_data: Optional[pd.DataFrame]`
* `adjust_args: Optional[dict]`

Let's `adjust` our predictor:

In [5]:
predictor.adjust(update_df, train_df)  # data to adjust and original data

INFO:dataprep_ml-2296:Cleaning the data
DEBUG:lightwood-2296: `preprocess` runtime: 0.03 seconds
INFO:dataprep_ml-2296:Cleaning the data
DEBUG:lightwood-2296: `preprocess` runtime: 0.01 seconds
INFO:dataprep_ml-2296:Updating the mixers
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:120: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
INFO:lightwood-2296:Loss @ epoch 1: 0.10915954659382503
DEBUG:lightwood-2296: `adjust` runtime: 0.16 seconds


In [6]:
new_predictions = predictor.predict(test_df)
new_predictions

INFO:dataprep_ml-2296:[Predict phase 1/4] - Data preprocessing
INFO:dataprep_ml-2296:Cleaning the data
DEBUG:lightwood-2296: `preprocess` runtime: 0.01 seconds
INFO:dataprep_ml-2296:[Predict phase 2/4] - Feature generation
INFO:dataprep_ml-2296:Featurizing the data
DEBUG:lightwood-2296: `featurize` runtime: 0.05 seconds
INFO:dataprep_ml-2296:[Predict phase 3/4] - Calling ensemble
INFO:dataprep_ml-2296:[Predict phase 4/4] - Analyzing output
INFO:lightwood-2296:The block ICP is now running its explain() method
INFO:lightwood-2296:The block ConfStats is now running its explain() method
INFO:lightwood-2296:ConfStats.explain() has not been implemented, no modifications will be done to the data insights.
INFO:lightwood-2296:The block AccStats is now running its explain() method
INFO:lightwood-2296:AccStats.explain() has not been implemented, no modifications will be done to the data insights.
INFO:lightwood-2296:The block PermutationFeatureImportance is now running its explain() method
INFO:

,original_index,prediction,confidence,lower,upper
0,0,43.645531,0.9991,0.000000,90.133981
1,1,26.964911,0.9991,0.000000,73.453361
2,2,24.151915,0.9991,0.000000,70.640365
3,3,20.815783,0.9991,0.000000,67.304233
4,4,34.987530,0.9991,0.000000,81.475980
...,...,...,...,...,...
201,201,52.630016,0.9991,6.141566,99.118467
202,202,39.175236,0.9991,0.000000,85.663686
203,203,33.047451,0.9991,0.000000,79.535902
204,204,28.659151,0.9991,0.000000,75.147602


Nice! Our predictor was updated, and new predictions are looking good. Let's compare the old and new accuracies to complete the experiment:

In [7]:
from sklearn.metrics import r2_score
import numpy as np

old_acc = r2_score(test_df['concrete_strength'], predictions['prediction'])
new_acc = r2_score(test_df['concrete_strength'], new_predictions['prediction'])

print(f'Old Accuracy: {round(old_acc, 3)}\nNew Accuracy: {round(new_acc, 3)}')

Old Accuracy: 0.233
New Accuracy: 0.428


## Conclusion

We have gone through a simple example of how Lightwood predictors can leverage newly acquired data to improve their predictions. The interface for doing so is fairly simple, requiring only some new data and a single call to update.

You can further customize the logic for updating your mixers by modifying the `partial_fit()` methods in them.